In [1]:
import pandas as pd
from datetime import datetime, timedelta

import math

import json as json
from pandas.io.json import json_normalize

import numpy as np
import os
import scipy.optimize as sco

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
array_dataframe = []
dados = pd.DataFrame([])
scoins = []

folder_candle = 'Operacoes'

for folder in os.scandir(folder_candle):
    scoins.append(folder.name)
    for entry in os.scandir(folder):
        if entry.is_file():
            coin_symbol = (os.path.splitext(entry.name)[0])        
            with open(entry.path) as d:            
                coin_hist = json.load(d);
            df = json_normalize(coin_hist);
            
            if not df.empty:
            
                df['coin'] = folder.name
                df['amount'] = pd.to_numeric(df['amount'].replace('.',','))
                df['rate'] = pd.to_numeric(df['rate'].replace('.',','))
                df['total'] = pd.to_numeric(df['total'].replace('.',','))
                df['date'] = pd.to_datetime(df['date'])            
                df['day'] = pd.to_datetime(df['date'])
                #df['day'] = pd.to_datetime(df['date'],format='%Y-%m-%d').dt.date
                
                array_dataframe.append(df)       
            else:
                print(entry.path)

dados = pd.concat(array_dataframe)

dados = dados.replace([np.inf, -np.inf], np.nan)
dados = dados.fillna(0)

dados.set_index('date', inplace=True)
dados.index = pd.to_datetime(dados.index)

dados.head()

Operacoes/USDT_BTC/2018-11.json
Operacoes/USDT_BTC/2018-12.json
Operacoes/USDT_ETH/2018-11.json
Operacoes/USDT_ETH/2018-12.json
Operacoes/USDT_LTC/2018-11.json
Operacoes/USDT_LTC/2018-12.json
Operacoes/USDT_ZEC/2018-11.json
Operacoes/USDT_ZEC/2018-12.json


,amount,globalTradeID,rate,total,tradeID,type,coin,day
date,,,,,,,,
2016-08-31 02:59:00,0.020843,51782713,577.106338,12.028627,687777,sell,USDT_BTC,2016-08-31 02:59:00
2016-08-31 02:58:46,0.020843,51782684,577.839987,12.043919,687776,buy,USDT_BTC,2016-08-31 02:58:46
2016-08-31 02:58:31,0.020843,51782678,577.106338,12.028627,687775,sell,USDT_BTC,2016-08-31 02:58:31
2016-08-31 02:58:18,0.020843,51782660,577.839988,12.043919,687774,buy,USDT_BTC,2016-08-31 02:58:18
2016-08-31 02:58:05,0.020843,51782633,577.106338,12.028627,687773,sell,USDT_BTC,2016-08-31 02:58:05


In [3]:
#Calcula a primeira data e a primeira janela de 24h

#data_inicial = dados.index.min()
data_inicial = pd.to_datetime('2015-08-01')
print(data_inicial)

#Calcula data final da primeira janela
data_final = data_inicial + timedelta(days=1)
print(data_final)

#Filtro para a janela
#mask_janela = (dados.index > data_inicial) & (dados.index <= data_final)

#Moeda base
moeda_base = 'USDT_BTC'

#Quantidade de ordens a serem avaliadas no head do book
qtd_relevante_head_book = 0.05 #5%
stop_inicial = 0.10 #10%

operacoes = []
minimo_analise = 5

id_operacoes = 0
#operacoes = dados
#operacoes['status'] = 'V'

2015-08-01 00:00:00
2015-08-02 00:00:00


In [4]:
def obtem_janela(dt_inicial, dt_final):
    return dados.loc[(dados.index > dt_inicial) & (dados.index <= dt_final)].sort_index(ascending=False)    

In [5]:
janela = obtem_janela(data_inicial,data_final)
janela.head(10)

,amount,globalTradeID,rate,total,tradeID,type,coin,day
date,,,,,,,,
2015-08-01 17:48:05,0.010000,9240823,279.845023,2.798450,19626,sell,USDT_BTC,2015-08-01 17:48:05
2015-08-01 17:43:27,0.000107,9240809,283.000000,0.030301,19625,buy,USDT_BTC,2015-08-01 17:43:27
2015-08-01 17:43:27,0.012905,9240808,279.850000,3.611372,19624,buy,USDT_BTC,2015-08-01 17:43:27
2015-08-01 17:38:59,0.224556,9240278,279.665000,62.800543,19623,buy,USDT_BTC,2015-08-01 17:38:59
2015-08-01 17:37:52,0.172513,9240246,275.724508,47.566018,19622,sell,USDT_BTC,2015-08-01 17:37:52
2015-08-01 17:37:49,0.360932,9240215,275.724508,99.517826,19621,sell,USDT_BTC,2015-08-01 17:37:49
2015-08-01 17:37:49,0.018569,9240214,278.000000,5.162249,19620,sell,USDT_BTC,2015-08-01 17:37:49
2015-08-01 17:37:49,0.000352,9240213,279.793314,0.098417,19619,sell,USDT_BTC,2015-08-01 17:37:49
2015-08-01 17:37:49,0.010130,9240212,279.793314,2.834320,19618,sell,USDT_BTC,2015-08-01 17:37:49


In [18]:
def calcula_qtd_relevante(dt):
    relevante = dt.coin.count() * qtd_relevante_head_book
    
    if relevante < minimo_analise:
        return minimo_analise
    
    return math.ceil(relevante)

def gera_id_operacao():
    global id_operacoes 
    id_operacoes = id_operacoes + 1
    return id_operacoes;

In [19]:
def avalia_moeda_base(dt):
    coin_result = dt[dt.coin == moeda_base]
    relevante = calcula_qtd_relevante(coin_result)
    
    if not coin_result.empty  :
        dt_sell = coin_result[coin_result.type == 'sell']
        dt_buy  = coin_result[coin_result.type == 'buy']
    
        #Verifica se a moeda base está em queda (sell > buy)
        if dt_sell.amount.sum() > dt_buy.amount.sum() :
            #print('Moeda base em queda')
            return True
    
    #print('Base não está em queda')
    return False


def avalia_compra(dt):
    
    #print(dt)        
    ## Verifica se a moeda base está no momento de compra
    if avalia_moeda_base(dt):
        
        for c in dt.coin.unique():
            coin_result = dt[dt.coin == c]
            
            relevante = calcula_qtd_relevante(coin_result)            
            
            coin_result = coin_result.head(relevante)
            
            if not coin_result.empty  :
                dt_sell = coin_result[coin_result.type == 'sell']
                dt_buy  = coin_result[coin_result.type == 'buy']
    
                #Verifica se a moeda base está em queda (sell > buy)
                if dt_buy.amount.sum() > dt_sell.amount.sum():
                    return True
    
    return False

In [25]:
def define_stop(dt):
    #print('Rate: '+ str(dt.rate))
    #print('10% Rate: '+ str((dt.rate * stop_inicial)))
    return dt.rate + (dt.rate * stop_inicial)

def simula_compra(dt):
    for c in dt[dt.coin != moeda_base].coin.unique() :        
        dt_moeda = dt[dt.coin == c]
        operacao_compra = dt[dt.coin == c].max()
        operacao_compra['id_operacao'] = gera_id_operacao()
        operacao_compra.type = 'buy'
        operacao_compra['closed'] = False
        operacao_compra['stop_update'] = 0
        operacao_compra['stop'] = define_stop(operacao_compra)
        operacoes.append(operacao_compra)
        #print('### COMPRA: Data/Hora:'+str(dt_moeda.day.max())+ ' Moeda: '+ str(c)+ ' Rate: '+str(dt_moeda[dt_moeda.coin == c].rate.max()))

In [29]:
def simula_venda(book_sell, operacao_buy):        
    #input("Vai Vender!")
    operacao_venda = book_sell[book_sell.coin == operacao_buy.coin].max()
    #print(book_sell)
    operacao_venda.type = 'sell'
    operacao_venda['id_operacao'] = operacao_buy.id_operacao
    operacao_buy['closed'] = True
    operacao_venda['closed'] = True
    operacoes.append(operacao_venda)
    
    #input("Vendeu!")
        
def estrategia(book):
    if not pd.DataFrame(operacoes).empty:
        
        for operacao_buy in operacoes:
            if (operacao_buy.type == 'buy') & (not operacao_buy.closed):
                coin_result = book[book.coin == operacao_buy.coin]               
                
                if not coin_result.empty  :
                    dt_buy = coin_result[coin_result.type == 'buy']
                    dt_buy = dt_buy[dt_buy.day > operacao_buy.day]
                    
                    relevante = calcula_qtd_relevante(dt_buy)
                    
                    dt_buy = dt_buy.head(relevante)
    
                    #Verifica se a moeda base está em queda (sell > buy)
                    if dt_buy.rate.max() <= operacao_buy.stop:
                        simula_venda(dt_buy, operacao_buy)
                    else:
                        #print('Stop Atual'+ str(operacao_buy.stop))
                        atualiza_stop(operacao_buy, coin_result)
                        #print('Stop Atualizado Para'+ str(operacao_buy.stop))

def atualiza_stop(operacao,book):
    #input('AtualizaStop')
    buy_book = book[book.type == 'buy']
    #print('BuyBook')
    #print(buy_book.head(500))
    last_order_price = buy_book[buy_book.coin == operacao.coin].mean()
    #print('last')
    #print(last_order_price.head(500))
    #input('Vai ser feito a atualização de um stop')
    if not buy_book.empty:
    #print(last_order_price)
        print('Stop Anterior:' + str(operacao.stop))
        operacao.stop_update = operacao.stop_update + 1
        operacao.stop = define_stop(last_order_price)
        print('Stop Novo:' + str(operacao.stop))
    #print('Preço Atualizado')
    #print(operacoes)

In [30]:
#RESET
operacoes = []
data_inicial = pd.to_datetime('2015-08-01')
data_final = data_inicial + timedelta(days=1)

#Coleta a Janela
janela = obtem_janela(data_inicial,data_final)

while not janela.empty :

    if avalia_compra(janela):
        simula_compra(janela)
    
    estrategia(janela)            
    
    data_inicial = data_final
    data_final = data_inicial + timedelta(days=1)
    
    #print('De: '+ str(data_inicial) + 'Até: '+ str(data_final))
    janela = obtem_janela(data_inicial,data_final)   
    #print(janela.head(100))
    #input("Próxima Janela")

Stop Anterior:5.225
Stop Novo:5.225
Stop Anterior:1.254
Stop Novo:1.02221405864
Stop Anterior:1.02221405864
Stop Novo:1.3375965845
Stop Anterior:1.3375965845
Stop Novo:1.81683333333
Stop Anterior:2.3649999999999998
Stop Novo:1.81683333333
Stop Anterior:1.81683333333
Stop Novo:2.3562
Stop Anterior:1.81683333333
Stop Novo:2.3562
Stop Anterior:2.419999989
Stop Novo:1.826
Stop Anterior:4.193537238
Stop Novo:5.225
Stop Anterior:1.7215
Stop Novo:1.671999989
Stop Anterior:2.97
Stop Novo:2.97
Stop Anterior:1.397
Stop Novo:1.397
Stop Anterior:2.97
Stop Novo:3.3769665215
Stop Anterior:3.596933043
Stop Novo:3.3769665215
Stop Anterior:1.397
Stop Novo:1.373997999
Stop Anterior:1.373997999
Stop Novo:1.4869079335
Stop Anterior:1.530099989
Stop Novo:1.4869079335
Stop Anterior:3.596933032
Stop Novo:3.596933032
Stop Anterior:1.429999406
Stop Novo:1.4299994005
Stop Anterior:1.4101990210000002
Stop Novo:1.2218796194
Stop Anterior:3.5969076110000002
Stop Novo:3.596907611
Stop Anterior:1.3089876249999999
St

Stop Anterior:4.66678179168
Stop Novo:6.07586352093
Stop Anterior:4.66678179168
Stop Novo:6.07586352093
Stop Anterior:4.66678179168
Stop Novo:6.07586352093
Stop Anterior:6.07586352093
Stop Novo:6.00892447714
Stop Anterior:6.07586352093
Stop Novo:6.00892447714
Stop Anterior:6.07586352093
Stop Novo:6.00892447714
Stop Anterior:6.47889
Stop Novo:5.96978154101
Stop Anterior:3.6299999450000002
Stop Novo:3.4914294987
Stop Anterior:6.2700000000000005
Stop Novo:5.69213341008
Stop Anterior:3.924656384
Stop Novo:3.73989143562
Stop Anterior:6.6
Stop Novo:6.28945345386
Stop Anterior:3.843173433
Stop Novo:3.70528857884
Stop Anterior:6.6
Stop Novo:6.48889815784
Stop Anterior:3.742617329
Stop Novo:3.60528193614
Stop Anterior:6.9289000000000005
Stop Novo:6.65737723453
Stop Anterior:3.7001729489999997
Stop Novo:3.53301691356
Stop Anterior:7.100500032999999
Stop Novo:6.90454636807
Stop Anterior:3.780017153
Stop Novo:3.7320651426
Stop Anterior:7.26
Stop Novo:7.12633820899
Stop Anterior:3.780017296
Stop No

Stop Anterior:9.4259011
Stop Novo:8.98416486846
Stop Anterior:58.54054162
Stop Novo:55.4965131535
Stop Anterior:5.1090324559999996
Stop Novo:5.00142325449
Stop Anterior:9.327999394999999
Stop Novo:9.07102420303
Stop Anterior:5.016
Stop Novo:4.84047755468
Stop Anterior:57.442000011
Stop Novo:55.5879372309
Stop Anterior:11.495
Stop Novo:11.3336989622
Stop Anterior:4.340594874
Stop Novo:4.27076960591
Stop Anterior:48.839999989
Stop Novo:48.1910328775
Stop Anterior:11.851814226999998
Stop Novo:11.6249017705
Stop Anterior:4.329049989
Stop Novo:4.29348674008
Stop Anterior:48.837333083
Stop Novo:48.1109012273
Stop Anterior:49.06
Stop Novo:48.4136323278
Stop Anterior:4.3164
Stop Novo:4.26992774427
Stop Anterior:12.091540164000001
Stop Novo:11.9064784842
Stop Anterior:48.487999988999995
Stop Novo:47.5379216825
Stop Anterior:11.929453305000001
Stop Novo:11.7998753849
Stop Anterior:4.263963682
Stop Novo:4.22784896506
Stop Anterior:11.847467698
Stop Novo:11.6609955642
Stop Anterior:47.85
Stop Novo

In [23]:
dt_operacoes = pd.DataFrame(operacoes)
dt_operacoes.sort_values('id_operacao').tail(60)

,amount,globalTradeID,rate,total,tradeID,type,coin,day,id_operacao,closed,stop_update,stop
587,2901.233316,75743188,10.774377,30898.134812,684305,buy,USDT_ETH,2017-01-20 23:57:27,841,True,1.0,11.851814
593,83.788000,75881293,11.000000,919.481927,686409,sell,USDT_ETH,2017-01-21 23:49:51,841,True,NaN,NaN
588,260.561914,75743158,3.935500,1020.393765,186949,buy,USDT_LTC,2017-01-20 23:57:16,842,True,1.0,4.329050
594,6.372048,75881613,3.922130,24.959314,187791,sell,USDT_LTC,2017-01-21 23:57:02,842,True,NaN,NaN
595,0.464141,75881401,43.935000,20.385064,138453,sell,USDT_ZEC,2017-01-21 23:52:26,843,True,NaN,NaN
589,16.591000,75742331,44.397576,721.454188,137446,buy,USDT_ZEC,2017-01-20 23:47:39,843,True,1.0,48.837333
596,41.218213,76016758,44.600000,1831.737199,139552,buy,USDT_ZEC,2017-01-22 23:58:45,844,True,1.0,49.060000
602,0.442447,76159129,42.992456,19.021884,140412,sell,USDT_ZEC,2017-01-23 23:53:04,844,True,NaN,NaN
603,5.000000,76159142,3.835000,19.121784,189460,sell,USDT_LTC,2017-01-23 23:53:15,845,True,NaN,NaN
597,230.046900,76016406,3.924000,898.091538,188608,buy,USDT_LTC,2017-01-22 23:56:50,845,True,1.0,4.316400


In [24]:
dt_operacoes['data'] = dt_operacoes.day
dt_operacoes.set_index('data', inplace=True)
dt_operacoes.tail(100)

,amount,globalTradeID,rate,total,tradeID,type,coin,day,id_operacao,closed,stop_update,stop
data,,,,,,,,,,,,
2016-11-26 23:59:20,3.675000,66416301,82.500000,298.326259,69736,sell,USDT_ZEC,2016-11-26 23:59:20,819,True,NaN,NaN
2016-11-27 23:59:04,3.981000,66559898,8.950994,35.315729,572451,sell,USDT_ETH,2016-11-27 23:59:04,820,True,NaN,NaN
2016-11-27 23:42:13,2.208137,66558116,73.936050,154.569588,71573,sell,USDT_ZEC,2016-11-27 23:42:13,821,True,NaN,NaN
2016-11-27 23:11:54,8.934000,66554343,3.851714,34.411216,141314,sell,USDT_LTC,2016-11-27 23:11:54,822,True,NaN,NaN
2016-11-30 01:58:30,0.023787,66842974,3.859584,0.091809,142633,buy,USDT_LTC,2016-11-30 01:58:30,823,True,1.0,4.245542
2016-11-30 01:54:29,98.818863,66842761,8.258297,797.468224,575867,buy,USDT_ETH,2016-11-30 01:54:29,824,True,1.0,9.084127
2016-11-30 01:54:26,2.271761,66842746,70.900000,155.868333,74052,buy,USDT_ZEC,2016-11-30 01:54:26,825,True,1.0,77.990000
2016-12-01 23:53:17,0.023861,67174808,3.912236,0.093350,143557,sell,USDT_LTC,2016-12-01 23:53:17,823,True,NaN,NaN
2016-12-01 23:45:39,3.777000,67173840,63.720430,234.551700,76262,sell,USDT_ZEC,2016-12-01 23:45:39,825,True,NaN,NaN
